In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from collections.abc import Sequence
import httpx
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
team_url = "https://www.transfermarkt.co.uk/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2021"

In [5]:
team_resp = httpx.get(
    team_url,
    headers={
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko gibi) Chrome/134.0.0.0 Safari/537.36'
    },
)

In [6]:
team_html = team_resp.content

In [7]:
page_soup = BeautifulSoup(team_html, "html.parser")

In [8]:
team_info = page_soup.find_all("td", {"class": "hauptlink no-border-links"})

In [9]:
team_name = [
    td.find("a").get("href").split("/")[1] if td.find("a") else None for td in team_info
]

In [10]:
team_id = [
    td.find("a").get("href").split("/")[4] if td.find("a") else None for td in team_info
]

In [11]:
urls = []
for td in team_info:
    data = td.find('a').get('href')
    team_name = data.split('/')[1]
    team_id = data.split('/')[4]
    year = data.split('/')[6]
    
    url = f'https://www.transfermarkt.co.uk/{team_name}/kader/verein/{team_id}/saison_id/{year}/plus/1'
    
    urls.append(url)
    

In [12]:
@dataclass
class Team:
    id: str
    name: str


class Parser(ABC):
    """ABC Protocol class for parsing data from transfermarkt."""

    @abstractmethod
    def parse(self, soup: BeautifulSoup) -> pd.DataFrame:
        pass


@dataclass
class Scraper:
    """Scrape data from transfermarkt for a given team and year."""

    team: Team
    parsers: Sequence[Parser]
    year: int
    url: str = (
        "https://www.transfermarkt.co.uk/{name}/kader/verein/{id}/saison_id/{year}/plus/1"
    )

    def run(self) -> pd.DataFrame:
        """Run the scraping process."""
        url = self.url.format(name=self.team.name, id=self.team.id, year=self.year)
        print(f"Scraping: {self.team.name} - {self.year}")

        soup = self._get_soup_content(url)  # get html content from url

        data = pd.concat(
            [parser.parse(soup) for parser in self.parsers], axis=1
        )  # concatenate parsers into a dataframe

        data["season"] = self.year  # add season to dataframe
        data["team"] = self.team.name  # add team name to dataframe

        return data

    def _get_soup_content(self, url: str) -> BeautifulSoup:
        """Get the html content from a given Transfermarkt url."""
        resp = self._make_request(url)
        return BeautifulSoup(resp.content, "html.parser")

    def _make_request(self, url: str) -> httpx.Response:
        """Make a request to a given Transfermarkt url."""
        try:
            response = httpx.get(
                url,
                headers={
                    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko gibi) Chrome/134.0.0.0 Safari/537.36'
                },
                timeout=60,
            )
            response.raise_for_status()
            return response

        except httpx.HTTPError as e:
            print(f"HTTP error occurred: {e}")
            raise e

In [13]:
class PlayerNames(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        elements = soup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
        names = [td.get("title") if td.get("title") else None for td in elements]
        return pd.Series(names, name="name")

In [14]:
class PlayerNumbers(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        numbers = [stat for stat in stats[0::8]]
        numbers = [
            (
                td.find("div", class_="rn_nummer").text.strip()
                if td.find("div", class_="rn_nummer")
                else None
            )
            for td in numbers
        ]
        return pd.Series(numbers, name="number")

In [15]:
class PlayerAges(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.DataFrame:
        stats = soup.find_all("td", {"class": "zentriert"})
        ages = [stat for stat in stats[1::8]]
        dob = [td.text.strip().split(" (")[0] if td.text else None for td in ages]
        age = [
            int(td.text.strip().split(" (")[1].split(")")[0]) if td.text else None
            for td in ages
        ]
        return pd.DataFrame({"dob": dob, "age": age})

In [16]:
class PlayerCountries(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        countries = [stat for stat in stats[2::8]]
        countries = [
            td.find("img").get("title") if td.find("img") else None for td in countries
        ]
        return pd.Series(countries, name="country")

In [17]:
class CurrentClubs(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        current_clubs = [stat for stat in stats[3::8]]
        current_clubs = [
            td.find("a").get("title") if td.find("a") else None for td in current_clubs
        ]
        return pd.Series(current_clubs, name="current_club")

In [18]:
class PlayerHeights(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        heights = [stat for stat in stats[4::8]]
        heights = [td.text if td.text else None for td in heights]
        return pd.Series(heights, name="height")

In [19]:
class PlayerFoot(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        foots = [stat for stat in stats[5::8]]
        foots = [td.text if td.text else None for td in foots]
        return pd.Series(foots, name="foot")

In [20]:
class PlayerJoinedDate(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        joined_date = [stat for stat in stats[6::8]]
        joined_date = [td.text if td.text else None for td in joined_date]
        return pd.Series(joined_date, name="joined_date")

In [21]:
class PlayerSigningFee(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        signing_info = [stat for stat in stats[7::8]]
        signing_fee = [
            td.find("a").get("title").split(": Ablöse ")[1] if td.find("a") else 0
            for td in signing_info
        ]
        return pd.Series(signing_fee, name="signing_fee")

In [22]:
class PlayerSignedFrom(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        signing_info = [stat for stat in stats[7::8]]
        signed_from = [
            td.find("a").get("title").split(": Ablöse ")[0] if td.find("a") else None
            for td in signing_info
        ]
        return pd.Series(signed_from, name="signed_from")

In [23]:
class PlayerValues(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        values = soup.find_all("td", {"class": "rechts hauptlink"})
        values = [td.find("a").text if td.find("a") else "€0" for td in values]
        return pd.Series(values, name="value")

In [24]:
class PlayerPositions(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        pos_soup = soup.find_all("td", {"class": "posrela"})
        positions = [
            td.find_all("tr")[1].find("td").text.strip() if td.find_all("tr") else None
            for td in pos_soup
        ]
        return pd.Series(positions, name="position")

In [25]:
class TransfermarktName(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        links = soup.find_all("td", {"class": "hauptlink"})
        tm_name = [
            link.find("a")["href"].split("/")[1] if link.find("a") else None
            for link in links[::2]
        ]
        return pd.Series(tm_name, name="tm_name")

In [26]:
class TransfermarktId(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        links = soup.find_all("td", {"class": "hauptlink"})
        tm_id = [
            link.find("a")["href"].split("/")[4] if link.find("a") else None
            for link in links[::2]
        ]
        return pd.Series(tm_id, name="tm_id")

In [27]:
parsers = (
    PlayerNames(),
    PlayerNumbers(),
    PlayerAges(),
    PlayerCountries(),
    CurrentClubs(),
    PlayerHeights(),
    PlayerFoot(),
    PlayerJoinedDate(),
    PlayerSigningFee(),
    PlayerSignedFrom(),
    PlayerValues(),
    PlayerPositions(),
    TransfermarktName(),
    TransfermarktId()
)

In [28]:
def get_team_info(league: str, league_id: str, year: int) -> tuple:
    link = "https://www.transfermarkt.co.uk/{league}/startseite/wettbewerb/{league_id}/plus/?saison_id={year}"
    url = link.format(league=league, league_id=league_id, year=year)
    resp = httpx.get(
        url,
        headers={
            "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko gibi) Chrome/134.0.0.0 Safari/537.36'
        },
        timeout=20,
    )
    soup = BeautifulSoup(resp.content, "html.parser")
    team_info = soup.find_all("td", {"class": "hauptlink no-border-links"})
    team_name = [td.find('a').get('href').split('/')[1] for td in team_info]
    team_id = [td.find('a').get('href').split('/')[4] for td in team_info]
    return tuple(zip(team_name, team_id))

In [29]:
pl_teams = get_team_info('premier-league', 'GB1', 2021)

In [30]:
teams = [Team(id=id, name=name) for name, id in zip(team_name, team_id)]

In [31]:
teams

[Team(id='1', name='f'),
 Team(id='1', name='c'),
 Team(id='3', name='-'),
 Team(id='2', name='b')]

In [32]:
dfs = []
for name, id_ in pl_teams:
    team = Team(id=id_, name=name)
    scraper = Scraper(team=team, parsers=parsers, year=2021)
    df = scraper.run()
    dfs.append(df)
    time.sleep(5) # sleep for 5 seconds to avoid getting blocked

Scraping: manchester-city - 2021
Scraping: fc-liverpool - 2021
Scraping: fc-chelsea - 2021
Scraping: manchester-united - 2021
Scraping: tottenham-hotspur - 2021
Scraping: fc-arsenal - 2021
Scraping: leicester-city - 2021
Scraping: aston-villa - 2021
Scraping: fc-everton - 2021
Scraping: wolverhampton-wanderers - 2021
Scraping: west-ham-united - 2021
Scraping: brighton-amp-hove-albion - 2021
Scraping: newcastle-united - 2021
Scraping: leeds-united - 2021
Scraping: fc-southampton - 2021
Scraping: fc-brentford - 2021
Scraping: crystal-palace - 2021
Scraping: norwich-city - 2021
Scraping: fc-watford - 2021
Scraping: fc-burnley - 2021


In [33]:
data = pd.concat(dfs)

In [34]:
data

,name,number,dob,age,country,current_club,height,foot,joined_date,signing_fee,signed_from,value,position,tm_name,tm_id,season,team
0,Ederson,31,"Aug 17, 1993",28,Brazil,Manchester City,"1,88m",left,"Jul 1, 2017",€40.00m,SL Benfica,€45.00m,Goalkeeper,ederson,238223,2021,manchester-city
1,Zack Steffen,13,"Apr 2, 1995",27,United States,Colorado Rapids,"1,91m",right,"Jul 9, 2019",€6.82m,Columbus Crew SC,€6.00m,Goalkeeper,zack-steffen,221624,2021,manchester-city
2,Scott Carson,33,"Sep 3, 1985",36,England,Manchester City,"1,88m",right,"Jul 20, 2021",free transfer,Derby County,€300k,Goalkeeper,scott-carson,14555,2021,manchester-city
3,Cieran Slicker,84,"Sep 15, 2002",19,Scotland,Ipswich Town,-,,None,0,None,€0,Goalkeeper,cieran-slicker,621997,2021,manchester-city
4,Rúben Dias,3,"May 14, 1997",25,Portugal,Manchester City,"1,87m",right,"Sep 29, 2020",€71.60m,SL Benfica,€75.00m,Centre-Back,ruben-dias,258004,2021,manchester-city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,Chris Wood,-,"Dec 7, 1991",30,New Zealand,Nottingham Forest,"1,91m",right,"Aug 21, 2017",€16.40m,Leeds United,€10.00m,Centre-Forward,chris-wood,108725,2021,fc-burnley
31,Jay Rodríguez,19,"Jul 29, 1989",32,England,Wrexham AFC,"1,85m",right,"Jul 9, 2019",€10.00m,West Bromwich Albion,€2.00m,Centre-Forward,jay-rodriguez,53360,2021,fc-burnley
32,Matej Vydra,27,"May 1, 1992",30,Czech Republic,FC Viktoria Plzen,"1,80m",right,"Aug 7, 2018",€12.20m,Derby County,€2.00m,Centre-Forward,matej-vydra,101500,2021,fc-burnley
33,Ashley Barnes,10,"Oct 30, 1989",32,England,Burnley FC,"1,85m",right,"Jan 2, 2025",free transfer,Norwich City,€1.50m,Centre-Forward,ashley-barnes,63200,2021,fc-burnley


In [35]:
# Export the data DataFrame to Excel
data.to_excel("../Datas/my_2021_premier_league_data.xlsx", index=False)